In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
# Input data files are available in the "../data/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../data"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
# load data
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values

id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values

# length of dataset
len_train = len(X_train)
len_test  = len(X_test)

test.csv
train.csv



In [5]:
for col in df_train.columns:
    print col

ID
var3
var15
imp_ent_var16_ult1
imp_op_var39_comer_ult1
imp_op_var39_comer_ult3
imp_op_var40_comer_ult1
imp_op_var40_comer_ult3
imp_op_var40_efect_ult1
imp_op_var40_efect_ult3
imp_op_var40_ult1
imp_op_var41_comer_ult1
imp_op_var41_comer_ult3
imp_op_var41_efect_ult1
imp_op_var41_efect_ult3
imp_op_var41_ult1
imp_op_var39_efect_ult1
imp_op_var39_efect_ult3
imp_op_var39_ult1
imp_sal_var16_ult1
ind_var1_0
ind_var1
ind_var5_0
ind_var5
ind_var6_0
ind_var6
ind_var8_0
ind_var8
ind_var12_0
ind_var12
ind_var13_0
ind_var13_corto_0
ind_var13_corto
ind_var13_largo_0
ind_var13_largo
ind_var13_medio_0
ind_var13
ind_var14_0
ind_var14
ind_var17_0
ind_var17
ind_var18_0
ind_var19
ind_var20_0
ind_var20
ind_var24_0
ind_var24
ind_var25_cte
ind_var26_0
ind_var26_cte
ind_var25_0
ind_var30_0
ind_var30
ind_var31_0
ind_var31
ind_var32_cte
ind_var32_0
ind_var33_0
ind_var33
ind_var34_0
ind_var37_cte
ind_var37_0
ind_var39_0
ind_var40_0
ind_var40
ind_var41_0
ind_var44_0
ind_var44
num_var1_0
num_var1
num_var4
num_var

In [8]:
rf = RandomForestClassifier(n_estimators=500)
rf.fit(X_train, y_train)
print "Features sorted by their score:"


Features sorted by their score:


In [10]:
var_imp = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), df_train.columns[1:]), 
             reverse=True)

In [13]:
imp_vars = []
non_imp_vars = 0
for varimp,var in var_imp:
    if varimp>0:
        imp_vars.append(var)


In [14]:
imp_vars

['var38',
 'var15',
 'saldo_medio_var5_ult3',
 'saldo_medio_var5_hace3',
 'num_var45_ult3',
 'saldo_var30',
 'saldo_medio_var5_hace2',
 'num_var45_hace3',
 'saldo_var42',
 'num_var45_hace2',
 'num_var22_ult3',
 'saldo_medio_var5_ult1',
 'saldo_var5',
 'num_var45_ult1',
 'num_var22_hace3',
 'num_var22_hace2',
 'num_med_var45_ult3',
 'var36',
 'num_var22_ult1',
 'num_meses_var39_vig_ult3',
 'num_meses_var5_ult3',
 'num_med_var22_ult3',
 'imp_op_var41_ult1',
 'imp_op_var39_ult1',
 'imp_op_var41_comer_ult3',
 'imp_op_var39_comer_ult3',
 'num_var35',
 'imp_trans_var37_ult1',
 'num_var4',
 'imp_op_var41_efect_ult3',
 'imp_op_var39_efect_ult3',
 'imp_op_var41_comer_ult1',
 'imp_op_var39_comer_ult1',
 'var3',
 'saldo_var37',
 'num_op_var41_ult3',
 'num_op_var39_ult3',
 'imp_op_var39_efect_ult1',
 'imp_op_var41_efect_ult1',
 'imp_ent_var16_ult1',
 'num_op_var41_comer_ult3',
 'num_op_var39_comer_ult3',
 'num_op_var41_ult1',
 'num_op_var39_hace2',
 'num_var30',
 'num_op_var41_hace2',
 'num_op_var

In [17]:
df_train=df_train[imp_vars]

In [18]:
correlations =  df_train.corr()

In [48]:
for i in range(0,len(imp_vars)-1):
    for j in range(i+1,len(imp_vars)):
        if correlations[imp_vars[i]][imp_vars[j]]>0.99:
            print imp_vars[i],"and",imp_vars[j],"are correlated with correlation:" , correlations[imp_vars[i]][imp_vars[j]]

num_var45_ult3 and num_med_var45_ult3 are correlated with correlation: 0.997806009995
imp_op_var41_ult1 and imp_op_var39_ult1 are correlated with correlation: 0.99110268492
imp_op_var41_efect_ult3 and imp_op_var39_efect_ult3 are correlated with correlation: 0.999270934254
imp_op_var39_efect_ult1 and imp_op_var41_efect_ult1 are correlated with correlation: 0.998378134203
num_op_var39_hace2 and num_op_var41_hace2 are correlated with correlation: 0.99259644747
num_op_var41_efect_ult3 and num_op_var39_efect_ult3 are correlated with correlation: 0.999244416638
num_op_var41_efect_ult1 and num_op_var39_efect_ult1 are correlated with correlation: 0.99896594905
num_var5 and ind_var5 are correlated with correlation: 0.99370918937
num_var8_0 and ind_var8_0 are correlated with correlation: 0.999793191936
saldo_var12 and saldo_var24 are correlated with correlation: 0.996932397338
num_var24_0 and ind_var24_0 are correlated with correlation: 0.997448385603
num_var40_0 and num_var1_0 are correlated wi

#pca

In [35]:
from sklearn import preprocessing
from sklearn.decomposition import PCA

pca = PCA(n_components=.99)

In [36]:
from sklearn import preprocessing
X_normalized = preprocessing.normalize(X_train, norm='l2')

In [37]:
x_pca = pca.fit_transform(X_normalized)

In [38]:
pca.explained_variance_

array([  5.91985703e-02,   1.87563839e-02,   8.12114346e-03,
         4.25923015e-03,   3.57316583e-03,   2.22360742e-03,
         1.45692967e-03,   1.34116785e-03,   1.12840309e-03,
         6.37991060e-04,   5.86282684e-04,   5.14901936e-04,
         4.46508613e-04,   4.24916968e-04,   4.17206222e-04,
         3.04049729e-04,   2.80740025e-04,   2.46933522e-04,
         2.26709042e-04,   2.14752935e-04,   1.95583816e-04,
         1.46629586e-04,   1.40411435e-04,   1.25520450e-04,
         1.22196583e-04,   9.32172270e-05])

In [41]:
rf = RandomForestClassifier(n_estimators=50)
rf.fit(x_pca,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [46]:
roc_auc_score(y_train, rf.predict_proba(x_pca)[:,1])

0.9220933338450048

In [47]:
X_normalized_test = preprocessing.normalize(X_test, norm='l2')
x_test_pca = pca.transform(X_normalized_test)

# predicting
y_pred= rf.predict_proba(x_test_pca)[:,1]

submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred})
submission.to_csv("submission_pca.csv", index=False)

print('Completed!')

Completed!
